In [19]:
# Importing Libraries
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
import pickle

In [3]:
# Importing Dataset as DataFrame
messages_df = pd.read_csv("smsspamcollection/SMSSpamCollection", sep = '\t',names = ['Label','Message'])

In [4]:
messages_df.head()

,Label,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [6]:
# Defining function for data cleaning and pre processing
def preprocess(message):
    corpus = []
    for i in range(len(message)):
        sentence = re.sub('[^a-zA-Z]',' ',message['Message'][i])
        sentence = sentence.lower()
        sentence = word_tokenize(sentence)
        sentence = [stemmer.stem(word) for word in sentence if word not in set(stopwords.words('english'))]
        sentence = " ".join(sentence)
        corpus.append(sentence)
    return corpus

In [7]:
corpus = preprocess(messages_df)
corpus[:20]

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri wkli comp win fa cup final tkt st may text fa receiv entri question std txt rate c appli',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl week word back like fun still tb ok xxx std chg send rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press copi friend callertun',
 'winner valu network custom select receivea prize reward claim call claim code kl valid hour',
 'mobil month u r entitl updat latest colour mobil camera free call mobil updat co free',
 'gon na home soon want talk stuff anymor tonight k cri enough today',
 'six chanc win cash pound txt csh send cost p day day tsandc appli repli hl info',
 'urgent week free membership prize jackpot txt word claim c www dbuk net lccltd pobox ldnw rw',
 'search right word thank breathe

In [8]:
# Creating the Bag of Words Model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000)
X = cv.fit_transform(corpus).toarray()

In [21]:
pickle.dump(cv,open('transform.pkl','wb'))

In [9]:
X.shape

(5572, 5000)

In [10]:
y = pd.get_dummies(messages_df["Label"])
y = y.iloc[:,1].values

In [11]:
# Train Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.20,
                                                    random_state = 0)

In [12]:
# Training model using Naive Bayes' Classifier
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
spam_detect_model = clf.fit(X_train, y_train)

In [13]:
y_pred = spam_detect_model.predict(X_test)

In [14]:
y_pred[:20]

array([0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
      dtype=uint8)

In [15]:
y_test[:20]

array([0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
      dtype=uint8)

In [16]:
spam_detect_model.score(X_test,y_test)

0.9838565022421525

In [17]:
from sklearn.metrics import confusion_matrix
confusion_m = confusion_matrix(y_test, y_pred)
confusion_m

array([[945,  10],
       [  8, 152]], dtype=int64)

In [18]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.9838565022421525

In [20]:
# Saving model to disk
pickle.dump(clf,open('nlp_model.pkl','wb'))